In [ ]:
!pip install flask line-bot-sdk

In [ ]:
import json
from random import choices, randint
from copy import deepcopy

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    PostbackEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


users = {}

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    t = event.message.text
    if t.isdigit() and 0 < int(t) < 6:
        with open('meactions.json', encoding='utf-8') as f:
            bubble = json.loads(f.read())
        carousel = {"type": "carousel", "contents": []}
        for i in range(int(t)):
            userid = 'U'+''.join([str(c) for c in choices(range(10), k=32)])
            users[userid] = {"count": randint(1, 1000)}
            b = deepcopy(bubble)
            b["body"]["contents"][0]["contents"][1]["text"] = userid
            b["footer"]["contents"][1]["action"]["data"] = userid
            carousel["contents"].append(b)
        msg = FlexMessage(altText='N Actions', contents=FlexContainer.from_dict(carousel))
    else:
        msg = TextMessage(text=t)
    
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[msg]
        )
    )

        
@handler.add(PostbackEvent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f"次數: {users[event.postback.data]['count']}")]
        )
    )

if __name__ == "__main__":
    app.run()